David Reed DS785 Spring 24

Classification BERT base uncased

 Set Runtime Type to T4 GPU for the lowest cost.

 Or to V100 High RAM

 Or to A100 High RAM

 but compute units are much more for the higher GPU, higher RAM options.



In [ ]:
! lscpu

Some problems with latest version of transformers so revert to version 4.31

In [ ]:
!pip install -q trl peft accelerate bitsandbytes datasets
!pip install -q git+https://github.com/huggingface/transformers@v4.31-release
!pip install -q torch
!pip install -q evaluate
!pip install -q huggingface_hub huggingface




In [ ]:
import os
os.getcwd()

#!pip list

In [ ]:
import numpy as np
import pandas as pd
import torch
import tensorflow as tf
from datasets import load_dataset, Dataset, DatasetDict, concatenate_datasets
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    T5Tokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    AutoConfig,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    AutoModelForSeq2SeqLM,
    Seq2SeqTrainingArguments,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainer,
    Trainer,
    BertTokenizer,
    BertModel,
    )
from peft import (
    LoraConfig,
    PeftModel,
    AutoPeftModelForCausalLM,
    prepare_model_for_kbit_training,
    get_peft_model,
    )
from trl import SFTTrainer
import sklearn
import matplotlib
import importlib.util
import pathlib
import importlib_metadata
import tensorflow as tf
import evaluate
%load_ext tensorboard
import huggingface_hub
from datetime import datetime
import nltk
import csv


Check GPU and usage.

In [ ]:
device= 'cpu'
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

In [ ]:


gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

torch.cuda.memory_allocated()

In [ ]:
time=datetime.now().strftime("%m-%d-%H-%M-%S")
str_time = str(time)
str_time

In [ ]:
# use the model from hugging face
model_base = "google-bert/bert-base-uncased"


# name for fine tuned model
model_fine_tuned = f"classBERTBase{str_time}"

# dataset directory
dataset_dir = "reeddg/retail_products_chatgtp"

#output directiry
dir_out = f"reeddg/training_outputs/{str_time}"

runs_log_dir = f"./{dir_out}"



In [ ]:
print (model_base, "\n", model_fine_tuned,"\n", dataset_dir, "\n", dir_out,"\n", runs_log_dir)

In [ ]:

# Load dataset from hugging face
dataset = load_dataset(dataset_dir)
# set review sentiment to class






In [ ]:
dft = dataset["train"].to_pandas()
#dft["text"] = dft[["Category", "Product Name", "User Review"]].apply(lambda x: x["Category"] + " " + x["User Review"] + " " + x["Product Name"] , axis=1)
dft["text"] = dft["User Review"]
dft["labels"] = dft["Review Sentiment"]
df = dft[['text' , 'labels']].copy()
dataset = Dataset.from_pandas(df)
dataset = dataset.class_encode_column("labels")
# stratify to have positive and negative reviews

dataset = dataset.train_test_split(test_size=0.3, stratify_by_column="labels")

In [ ]:
dataset


In [ ]:
dataset["train"]

In [ ]:
dfr = dataset["train"].to_pandas()
counts = dfr['labels'].value_counts()
print("Train\n", counts)
dfr = dataset["test"].to_pandas()
counts = dfr['labels'].value_counts()
print("Test\n", counts)

In [ ]:
dataset["train"][0]

In [ ]:
dataset["train"].to_pandas()

In [ ]:
#Tokenizer
#Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_base, use_fast=True, add_prefix_space=True)


In [ ]:
# NOT add pad token if none exists
  #if tokenizer.pad_token is None:
   # tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    # model.resize_token_embeddings(len(tokenizer))

In [ ]:
# create tokenize function
def tokenize_function(examples):
    # extract text
    text = examples["text"]

    #tokenize and truncate text
    tokenizer.truncation_side = "left"
    tokenized_inputs = tokenizer(
        text,
        return_tensors="np",
        truncation=True,
        max_length=512
    )

    return tokenized_inputs

In [ ]:
# tokenize training and validation datasets
tokenized_dataset = dataset.map(tokenize_function, batched=True)
tokenized_dataset

In [ ]:
# create data collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
data_collator

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)


# define label maps
id2label = {0: "Negative", 1: "Positive"}
label2id = {"Negative":0, "Positive":1}

# Load base moodel
model = AutoModelForSequenceClassification.from_pretrained(model_base, num_labels=2, id2label=id2label, label2id=label2id, quantization_config=bnb_config,
    device_map={"": 0})



model.config.use_cache = False
model.config.pretraining_tp = 1

In [ ]:
model

## Create Evaluation

In [ ]:
# accuracy evaluation
accuracy = evaluate.load("accuracy")
recall = evaluate.load("recall")
precision = evaluate.load("precision")
f1 = evaluate.load("f1")

In [ ]:
# eval fxn for trainer
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=1)
    a = accuracy.compute(predictions=predictions, references=labels)
    r = recall.compute(predictions=predictions, references=labels)
    p = precision.compute(predictions=predictions, references=labels)
    f = f1.compute(predictions=predictions, references=labels)
    return {"accuracy": a['accuracy'], "precision": p['precision'], "recall": r['recall'], "f1": f['f1']}

In [ ]:
# define list of examples
text_list = ["It was good.", "Not a fan, don't recommed.", "Better than the first one.", "This is not worth watching even once.", "This one is a pass."]

print("Untrained model predictions:")
print("----------------------------")
for text in text_list:
    # tokenize text
    inputs = tokenizer.encode(text, return_tensors="pt")
    # compute logits
    logits = model(inputs).logits
    # convert logits to label
    predictions = torch.argmax(logits)

    print(text + " - " + id2label[predictions.tolist()])

In [ ]:
peft_config = LoraConfig(
        r=8, lora_alpha=16, lora_dropout=0.05, task_type="SEQ_CLS"
    )

In [ ]:
peft_config

In [ ]:
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

In [ ]:
training_arguments = TrainingArguments(
        output_dir= runs_log_dir,
        per_device_train_batch_size=4,
        gradient_accumulation_steps=1,
        optim="paged_adamw_8bit",
        learning_rate=2e-4,
        lr_scheduler_type="linear",
        save_strategy="steps",
        logging_steps=10,
        num_train_epochs=3,
        report_to="tensorboard",
        max_steps=-1,
        push_to_hub=True,
        evaluation_strategy="steps",
        load_best_model_at_end=True,
    )

In [ ]:
trainer = Trainer(
        model=model,
        args=training_arguments,
        tokenizer=tokenizer,
        train_dataset=tokenized_dataset["train"],
        eval_dataset=tokenized_dataset["test"],
        data_collator=data_collator,
        compute_metrics=compute_metrics,
    )

In [ ]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    print("Name:", gpu.name, "  Type:", gpu.device_type)

gpus


visible_devices = tf.config.get_visible_devices()
for devices in visible_devices:
  print(devices)

tf.config.list_physical_devices('GPU')

In [ ]:
# Train model
trainer.train()

In [ ]:
logdir_path = f"{dir_out}/runs"
print (logdir_path)
f"{dir_out}"

In [ ]:
logdir_path = f"{dir_out}/runs"
print (logdir_path)
#huggingface_hub.upload_folder
#/content/reeddg/training_outputs3/runs/Mar24_14-57-33_ecd96771620a
%load_ext tensorboard
#%tensorboard --logdir reeddg/training_outputs/04-07-05-45-10
%tensorboard --logdir "$dir_out"

In [ ]:
# save fine-tuned model
ft_model_path = f"reeddg/test_train/class_{model_fine_tuned}"
trainer.model.save_pretrained(ft_model_path)
trainer.save_model(ft_model_path)
trainer.push_to_hub(ft_model_path)

Create model on HuggingFace first

In [ ]:
# reload base model back in
base_model= AutoModelForSequenceClassification.from_pretrained(model_base, num_labels=2)



In [ ]:
# merge base model and the fine-tuned model
merged_model= PeftModel.from_pretrained(base_model, ft_model_path)
merged_model= merged_model.merge_and_unload()

In [ ]:
# save merged model
merged_model_path= f"reeddg/test_model/merged_{model_fine_tuned}"
merged_model.save_pretrained(merged_model_path)

In [ ]:
merged_model.push_to_hub("reeddg/bert_test")
tokenizer.push_to_hub("reeddg/bert_test")

In [ ]:
# define list of examples
text_list = ["The cleaner did not work well.", "I do not recommed.", "Tore and ripped.", "Very Good.", "This one fit well."]

print("Trained model predictions:")
print("----------------------------")
for text in text_list:
    # tokenize text
    inputs = tokenizer.encode(text, return_tensors="pt")
    # compute logits
    logits = merged_model(inputs).logits
    # convert logits to label
    predictions = torch.argmax(logits)

    print(text + " - " + id2label[predictions.tolist()])

In [ ]:
# define list of examples
text_list = ["not good.", "Very good.", "faded", "Very Strong.", "Fit Great."]

print("Untrained model predictions:")
print("----------------------------")
for text in text_list:
    # tokenize text
    inputs = tokenizer.encode(text, return_tensors="pt")
    # compute logits
    logits = merged_model(inputs).logits
    # convert logits to label
    predictions = torch.argmax(logits)

    print(text + " - " + id2label[predictions.tolist()])
    print(predictions.tolist())

In [ ]:
original_model =AutoModelForSequenceClassification.from_pretrained(model_base, num_labels=2, id2label=id2label, label2id=label2id, quantization_config=bnb_config,
    device_map={"": 0})


In [ ]:
eval_tokenizer = AutoTokenizer.from_pretrained("reeddg/bert_test", use_fast=True)
#eval_tokenizer.pad_token = eval_tokenizer.eos_token

def gen(model,p, maxlen=68, sample=True):
    toks = eval_tokenizer(p, return_tensors="pt")
    #res = model.generate(**toks.to("cuda"), max_new_tokens=maxlen, do_sample=sample,num_return_sequences=1,temperature=0.1,num_beams=1,top_p=0.95,).to('cuda')
    res = model.generate(**toks.to("cuda"),max_new_tokens=maxlen,num_return_sequences=1,penalty_alpha=0.6,do_sample = True,
      top_k=5,temperature=0.5,repetition_penalty=1.2,).to('cuda')
    return eval_tokenizer.batch_decode(res,skip_special_tokens=True)




In [ ]:
from time import perf_counter

qty_to_test=len(dataset['test'])
dialogues = dataset['test'][0:qty_to_test]['text']
baseline_ref = dataset['test'][0:qty_to_test]['labels']


print(dialogues)
print(baseline_ref)

original_ref = []
original_model_predict = []
peft_model_predict = []
time_to_respond_orig = []
time_to_respond_peft = []

for idx, dialogue in enumerate(dialogues):
    baseline_ref_output = baseline_ref[idx]


    prompt = f"{dialogue}"
    print(prompt)
    inputs = eval_tokenizer.encode(dialogue, return_tensors="pt")

    start_time=perf_counter()


    # compute logits
    logits = original_model(inputs).logits
    # convert logits to label
    predictions_o = torch.argmax(logits)
    original_model_pred= (predictions_o.tolist())
    print(original_model_pred)

    out_time = perf_counter() - start_time
    time_to_respond_orig.append(out_time)
    print(f"Time taken for original inference: {round(out_time,2)} seconds")

    start_time=perf_counter()


    #inputs = tokenizer.encode(dialogue, return_tensors="pt")
    # compute logits
    logits = merged_model(inputs).logits
    # convert logits to label
    predictions_m = torch.argmax(logits)
    peft_model_pred= (predictions_m.tolist())
    print(peft_model_pred)

    out_time = perf_counter() - start_time
    time_to_respond_peft.append(out_time)
    print(f"Time taken for peft inference: {round(out_time,2)} seconds")

    original_model_predict.append(original_model_pred)
    peft_model_predict.append(peft_model_pred)

zipped_summaries = list(zip(baseline_ref, original_model_predict, peft_model_predict))
zipped_time = list(zip(time_to_respond_orig, time_to_respond_peft))

df = pd.DataFrame(zipped_summaries, columns = ['baseline_ref', 'original_model_predict', 'peft_model_predict'])
df_time = pd.DataFrame(zipped_time, columns = ['orig_model_inference_time', 'peft_model_inference_time'])
df

In [ ]:
def compare_results(results):
    ref = results['baseline_ref']
    p_pred = results['peft_model_predict']
    o_pred = results['original_model_predict']

    peft_a = accuracy.compute(predictions=p_pred, references=ref)
    peft_r = recall.compute(predictions=p_pred, references=ref)
    peft_p = precision.compute(predictions=p_pred, references=ref)
    peft_f = f1.compute(predictions=p_pred, references=ref)
    p = pd.array([peft_a['accuracy'], peft_r['recall'], peft_p['precision'], peft_f['f1']])

    o_a = accuracy.compute(predictions=o_pred, references=ref)
    o_r = recall.compute(predictions=o_pred, references=ref)
    o_p = precision.compute(predictions=o_pred, references=ref)
    o_f = f1.compute(predictions=o_pred, references=ref)
    o = pd.array([o_a['accuracy'], o_r['recall'], o_p['precision'], o_f['f1']])

    ret = pd.DataFrame([p,o],columns=["accuracy", "recall", "precision","f1"], index=["peft", "original"])
    return(ret)


dataf=compare_results(df)
print(dataf)

In [ ]:

print (f"Inference time peft: {round(np.average(time_to_respond_peft),4)}")
print (f"Inference time original: {round(np.average(time_to_respond_orig),4)}")

time_p2o = np.subtract(time_to_respond_peft, time_to_respond_orig)
time_2inf = np.average(time_p2o)
time_p=["Time2Imp"]
#print(time_p)
time_p = [*time_p, *time_p2o]
#print (time_p)


if(time_2inf>0):
  print(f"Inference time for original model is on average {round(time_2inf,2)} seconds better than the peft model.")
else:
  print(f"Inference time for peft model is on average {round(-time_2inf,2)} seconds better than the original model.")

In [ ]:
file= model_fine_tuned + "class.csv"

dataf.to_csv(file, encoding='utf-8')

file_time= "time_" + file

df_time.to_csv(file_time,encoding='utf-8')

